In [1]:
!pip install git+https://github.com/huggingface/transformers -q

In [2]:
!pip install qwen-vl-utils -q


In [3]:
from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor


/Users/rupinajay/.local/share/virtualenvs/Amazon_ML_Challenge_24-U8Zx5Tr6/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
from PIL import Image
import requests

In [7]:
model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

ImportError: Using `low_cpu_mem_usage=True` or a `device_map` requires Accelerate: `pip install 'accelerate>=0.26.0'`

In [ ]:
image = Image.open("21iGjAhB0ML.jpg")

messages = [
    {
        "role": "user",
        "content": [
            {
                "type": "image",
            },
            {
                "type": "text",
                "text": "Extract entities perfectly from the image and give the output in a json format only."
            }
        ]
    }
]

text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

inputs = processor(
    text = [text_prompt],
    images = [image],
    padding = True,
    return_tensors = "pt"
)

inputs = inputs.to("cuda")

output_ids = model.generate(**inputs, max_new_tokens=1024)

generated_ids = [
    output_ids[len(input_ids) :]
    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
]

output_text = processor.batch_decode(
    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
)

In [ ]:
print(output_text)

['```json\n{\n  "entities": [\n    {\n      "type": "length",\n      "value": "7.4 in",\n      "unit": "in",\n      "description": "The length of the peeler is 7.4 inches."\n    },\n    {\n      "type": "length",\n      "value": "18 cm",\n      "unit": "cm",\n      "description": "The length of the peeler is 18 centimeters."\n    }\n  ]\n}\n```']
